# Au delà de numpy : numba
* Calculer $\pi$ (avec une formule très très lente!!!)
$$ \frac\pi4 = \sum_i \frac{(-1)^i}{2i+1} = 1 - \frac13 + \frac 15 - \frac17 + \ldots $$

* numba.vectorize


In [1]:
def pi_python(N):
    res = 0
    coef = 1
    for i in range(N):
        res += coef/(2*i+1)
        coef = -coef
    return 4*res

#%timeit pi_python(1000000) # 77 ms

In [5]:
import numpy as np
def pi_np(N):
    Ti = np.arange(N)
    moins_un_puissance_i = ( 1-2*(Ti%2) )
    return 4*np.sum( moins_un_puissance_i/(2*Ti+1))

%timeit pi_np(1000000) # 11.5ms

30.1 ms ± 4.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
from numba import jit, int64, float64
import numba
# Just in time complier

numba_pi = jit( float64(int64) )(pi_python)

%timeit numba_pi(1000000)

909 µs ± 42.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
numba_pi(1000000)

3.1415916535897743

In [5]:
# Décorateur @
@jit( float64(int64) )
def numba_pi(N):
    res = 0
    coef = 1
    for i in range(N):
        res += coef/(2*i+1)
        coef = -coef
    return 4*res


In [10]:
#@jit(float64[:](float64[:], float64[:]), parallel=True)
@jit(parallel=True)
def somme(a, b):
    N = len(a)
    c = np.zeros(N)
    for i in numba.prange(N):
        c[i] = (a[i] + b[i])**2 + np.sin(a[i])
    return c


a = np.linspace(1, 2, 1000000)
b = np.logspace(1, 2, 1000000)

somme(a, b)

%timeit somme(a, b)

2.33 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit (a + b)**2 + np.sin(a)

3.9 ms ± 331 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# Numpy utilise des variables intermédiaires -> vitesse limitée par la mémoire
#(a + b)*c + a
#tmp1 = a + b
#tmp2 = tmp1*c
#res = tmp2 + a

In [12]:
def volume(dimension, M):
    pts = np.random.rand(M, dimension)*2 - 1
    carre_norme = np.sum(pts**2, axis=1)
    return np.mean(carre_norme<1) * 2**dimension

volume(5, 1000000)

5.269632

In [13]:
%timeit volume(10, 10000000)

2.79 s ± 144 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
@jit(float64(float64, int64))
def volume_numba(dimension, M):
    output = 0
    for i in range(M):
        point = np.random.rand(dimension)*2 -1 
        carre_norme = np.sum(point**2)
        if carre_norme<1:
            output += 1
    return output/M * 2**dimension


volume_numba(5, 1000000)


5.281152

In [15]:
%timeit volume_numba(10, 10000000)

1.71 s ± 6.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
